Source Code for **Automated LCI Compilation**

Import libraries

In [1]:
import pandas as pd
import json
import numpy as np
import openpyxl as opx

Load data structure with WWTP data

In [7]:
with open("data_structure.json","r") as main:
    wwtp_data = json.load(main)

Create DataFrame that contains material, material type, quantity and unit [1] for input and output site,<br>
materials obtained from [1] ecoinvent 3.8 database by ecoinvent Association 2021,<br>
(except user generated materials: wastewater influent, sewage sludge wet, screenings, grit chamber trappings, wastewater effluent)

In [3]:
wastewater_treatment_df = pd.DataFrame(
{
    "mat_in": [
        "wastewater influent",
        "electricity, medium voltage",
        "natural gas, low pressure",
        "light fuel oil",
        "aluminium sulfate, without water, in 4.33% aluminium solution state",
        "iron(III) sulfate, without water, in 12.5% iron solution state",
        "iron(III) chloride, without water, in a 12% iron solution state",
        "iron(II) chloride",
        "sodium aluminate, powder",
        "polyaluminium chloride",
        " ",
        " ",
        " ",
        " "
    ],
    "type_in": [
        "Bad",
        "Good",
        "Good",
        "Good",
        "Good",
        "Good",
        "Good",
        "Good",
        "Good",
        "Good",
        "Good",
        "Good",
        "Good",
        "Good"
    ],
    "quan_in": [
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0
    ],
    "unit_in":[
        "m³",
        "kWh",
        "m3",
        "kg",
        "kg",
        "kg",
        "kg",
        "kg",
        "kg",
        "kg",
        " ",
        " ",
        " ",
        " "
    ],
    "mat_out": [
        "BOD5, Biological Oxygen Demand [water/surface water]",
        "COD, Chemical Oxygen Demand [water/surface water]",
        "Ammonium, ion [water/surface water]",
        "Nitrate [water/surface water]",
        "Nitrite [water/surface water]",
        "Phosphorus [water/surface water]",
        "Methane, non-fossil [air/unspecified]",
        "Dinitrogen monoxide [air/unspecified]",
        "sewage sludge wet",
        "sewage sludge wet",
        "sewage sludge wet",
        "screenings",
        "grit chamber trappings",
        "wastewater effluent"
    ],
    "type_out": [
        "Bad",
        "Bad",
        "Bad",
        "Bad",
        "Bad",
        "Bad",
        "Bad",
        "Bad",
        "Bad",
        "Bad",
        "Bad",
        "Bad",
        "Bad",
        "Neutral"
    ],
    "quan_out": [
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0
    ],
    "unit_out":[
        "kg",
        "kg",
        "kg",
        "kg",
        "kg",
        "kg",
        "kg",
        "kg",
        "t",
        "t",
        "t",
        "t",
        "t",
        "m³"
    ]
})

Overwrite empty entries with NaN

In [4]:
wastewater_treatment_df.iloc[10:14, 0:4] = np.nan

List with names of precipitants

In [5]:
precipitants = ['aluminium sulfate', 'iron(III) sulfate',
                'iron(III) chloride', 'iron(II) chloride',
                'sodium aluminate', 'polyaluminium chloride']

Function: Calculate percentage of each sludge disposal path and write to list

In [6]:
def disposal_path_percentage_calc(disposal_dict):

    path_sum = 0
    path_list = []

    for key in disposal_dict.keys():
        if key == 'unit':
            continue
        if key == 'location':
            break

        
        path_sum = path_sum + wwtp_data['sludge disposal'][key]

        path_list.append(wwtp_data['sludge disposal'][key])

      
    percentage_list = []
    for item in range(len(path_list)):
        percentage_list.append(path_list[item] / path_sum)
    
    return(percentage_list)

**Write values from WWTP data structure to DataFrame wastewater_treatment_df**

In [8]:
#Input
wastewater_treatment_df.at[0, 'quan_in'] = wwtp_data['influent']['wastewater_influent']
wastewater_treatment_df.at[1, 'quan_in'] = wwtp_data['energy']['electricity_from_grid']['value']
wastewater_treatment_df.at[2, 'quan_in'] = wwtp_data['energy']['natural_gas_consumption']['value']
wastewater_treatment_df.at[3, 'quan_in'] = wwtp_data['energy']['fuel_oil_consumption']['value']

#precipitant
if wwtp_data['plant']['phosphorus_eliminiation'] == True:
    num = 0
    for i in range(6):
        if wwtp_data['plant']['phosphorus_precipitant'] == precipitants[i]:
            num = i + 4
            wastewater_treatment_df.at[num, 'quan_in'] = wwtp_data['plant']['phosphorus_precipitant_consumption']['value']
#Output
#effluent [2]
wastewater_treatment_df.at[0, 'quan_out'] = wwtp_data['effluent']['BOD'] * \
(wwtp_data['influent']['wastewater_influent'] - wwtp_data['sludge']['volume']['value']) * 10**(-3)
wastewater_treatment_df.at[1, 'quan_out'] = wwtp_data['effluent']['COD'] * \
(wwtp_data['influent']['wastewater_influent'] - wwtp_data['sludge']['volume']['value']) * 10**(-3)
wastewater_treatment_df.at[2, 'quan_out'] = wwtp_data['effluent']['NH4-N'] * \
(wwtp_data['influent']['wastewater_influent'] - wwtp_data['sludge']['volume']['value']) * 10**(-3) * 1.29
wastewater_treatment_df.at[3, 'quan_out'] = wwtp_data['effluent']['NO3-N'] * \
(wwtp_data['influent']['wastewater_influent'] - wwtp_data['sludge']['volume']['value']) * 10**(-3) * 4.43
wastewater_treatment_df.at[4, 'quan_out'] = wwtp_data['effluent']['NO2-N'] * \
(wwtp_data['influent']['wastewater_influent'] - wwtp_data['sludge']['volume']['value']) * 10**(-3) * 3.28
wastewater_treatment_df.at[5, 'quan_out'] = wwtp_data['effluent']['Ptot'] * \
(wwtp_data['influent']['wastewater_influent'] - wwtp_data['sludge']['volume']['value']) * 10**(-3)

#GHG emissions[3]
#methane
wastewater_treatment_df.at[6, 'quan_out'] = wwtp_data['effluent']['BOD'] * \
(wwtp_data['influent']['wastewater_influent'] - wwtp_data['sludge']['volume']['value']) * 10**(-3) * 0.068

#nitrous oxide
#n2o treatment
if wwtp_data['influent']['totN'] is None:
    if wwtp_data['influent']['NH4-N'] is None:
        totN_inf = wwtp_data['influent']['COD'] * (5/48) #[4,5] cited in [6]
    else:
        totN_inf = wwtp_data['influent']['NH4-N'] * 1.7 #[7]
else:
    totN_inf = wwtp_data['influent']['totN']

n2o_inf = totN_inf * wwtp_data['influent']['wastewater_influent'] * 10**(-3) * 0.016 * 44/28

#n20 effluent
if wwtp_data['effluent']['totN'] is None:
    totN_eff = wwtp_data['effluent']['Ntot'] + 2 #[7]
else:
    totN_eff = wwtp_data['effluent']['totN']

n2o_eff = (totN_eff * (wwtp_data['influent']['wastewater_influent'] - wwtp_data['sludge']['volume']['value']) * \
10**(-3) * 0.005 * 44/28)

n2o = n2o_inf + n2o_eff

wastewater_treatment_df.at[7, 'quan_out'] = n2o

#sludge disposal paths
#deploy function: list of percentages of different disposal paths
disposal_list = disposal_path_percentage_calc(wwtp_data['sludge disposal'])

#sewage sludge with no dewatering: agriculture, humification
wastewater_treatment_df.at[8, 'quan_out'] = disposal_list[0] * wwtp_data['sludge']['volume']['value']

wastewater_treatment_df.at[9, 'quan_out'] = disposal_list[5] * wwtp_data['sludge']['volume']['value']

#sewage sludge to dewatering: landsacping, incineration, landfilling, recultivation
wastewater_treatment_df.at[10, 'quan_out'] = (disposal_list[1] +
                                             disposal_list[2] +
                                             disposal_list[3] +
                                             disposal_list[4] +
                                             disposal_list[6]) * wwtp_data['sludge']['volume']['value']



#screenings[8]
if wwtp_data['sludge']['screenings']['value'] == None:
    wastewater_treatment_df.at[11, 'quan_out'] = wwtp_data['influent']['load'] * 1.4 * 10**(-3)
else:
    wastewater_treatment_df.at[11, 'quan_out'] = wwtp_data['sludge']['screenings']['value']
    
#grit champber trappings
wastewater_treatment_df.at[12, 'quan_out'] = wwtp_data['sludge']['grit_chamber_trappings']['value']

#effluent
wastewater_treatment_df.at[13, 'quan_out'] = wwtp_data['influent']['wastewater_influent'] - \
wwtp_data['sludge']['volume']['value']

[2] The factors used to transform NH4-N, NO3-N and NO2-N into NH4, NO3, and NO2 are obtained from Landesamt für Natur, Umwelt und Verbraucherschutz Nordrhein-Westfalen, https://www.abfall-nrw.de/abanda/help/1_umrechnung.htm

[3] The emissions of methane and nitrous oxide are calculated according to the guidelines of the IPCC, using the emission factors of the IPCC, 2019 Refinement to the 2006 IPCC Guidelines for National Greenhouse Gas Inventories: Volume 5: Waste,
Chapter 6: WASTEWATER TREATMENT AND DISCHARGE, https://www.ipcc-nggip.iges.or.jp/public/2019rf/vol5.html

[4] Gehring, T. ; Deineko, E. ; Hobus, I. ; Kolisch, G. ; Lübken, M. ; Wichern, M.: Effect of Sewage Sampling Frequency on Determination of Design Parameters for Municipal Wastewater Treatment Plants. In: Water science and technology : a journal of the International Association on Water Pollution Research 84 (2021), Nr. 2, S. 284–292. http://dx.doi.org/10.2166/wst.2020.588 – DOI 10.2166/wst.2020.588. – ISSN 0273–1223

[5] Hobus, I. ; Gehring, T. ; Deineko, E. ; Wichern, M. ; Kolisch, G.:Belastungsermittlung Für Die Bemessung von Einstufigen Belebungsanlagen Nach Dem Arbeitsblatt DWA-A 131. In: KA Korrespondenz Abwasser, Abfall 69 (2022), Nr. 3, S. 207–214

[6] calculation of totN from COD is obtained from Arbeitsblatt DWA-A 198 Ermittlung von Bemessungswerten Für Abwasseranlagen (Entwurf). Februar 2022. Hennef : Deutsche Vereinigung für Wasserwirtschaft, Abwasser und Abfall, 2022. ISBN 978–3–96862–197–5

[7] factors to calculate totN (influent) from NH4-N and totN (effluent) from Ntot
is obtained from DWA-Landesverband Bayern, Kanal- und Kläranlagen-Nachbarschaften: DWA-Leistungsvergleich, Anlage Mit Vorklärung 2022. https://www.dwa-bayern.de/files/_media/content/PDFs/LV_Bayern/5%20Nachbarschaften/Leistungsvergleich%202022_Anlage%20mit%20VK.xls

[8] The factor to calculate the amount of screenings is obtained from Kuhn M., Gregor H. 2013. Rechengutanfall und -qualität. Korrespondenz Abwasser. 60 (2013), Nr. 1, S. 30–36

**LCI is now in DataFrame wastewater_treatment_df**

In [1]:
wastewater_treatment_df

**Transfer values of Dataframe to xlsx template**

In [10]:
wb = opx.load_workbook("templ.xlsx")
ws = wb.active
#input
for i in range(10):
    ws.cell(row = i + 4, column = 1, value = wastewater_treatment_df["mat_in"][i])
    ws.cell(row = i + 4, column = 4, value = wastewater_treatment_df["type_in"][i])
    ws.cell(row = i + 4, column = 6, value = wastewater_treatment_df["quan_in"][i])
    ws.cell(row = i + 4, column = 7, value = wastewater_treatment_df["unit_in"][i])
#output
for i in range(14):
    ws.cell(row = i + 4, column = 9, value = wastewater_treatment_df["mat_out"][i])
    ws.cell(row = i + 4, column = 12, value = wastewater_treatment_df["type_out"][i])
    ws.cell(row = i + 4, column = 14, value = wastewater_treatment_df["quan_out"][i])
    ws.cell(row = i + 4, column =15, value = wastewater_treatment_df["unit_out"][i])
wb.save("wastewater_treatment.xlsx")